In [1]:
import json

with open('train.jsonl', 'r') as json_file:
    json_list = list(json_file)
question_infors = []
for json_str in json_list:
    result = json.loads(json_str)
    question_infors.append(result)


with open('dev.jsonl', 'r') as json_file:
    json_list = list(json_file)
dev = []
for json_str in json_list:
    result = json.loads(json_str)
    dev.append(result)

with open('test.jsonl', 'r') as json_file:
    json_list = list(json_file)
test = []
for json_str in json_list:
    result = json.loads(json_str)
    test.append(result)

In [2]:
question_infors[0]

{'qID': '3UZUVSO3P99H5YO9049ZTR528ADMEV-1',
 'sentence': "The boy got in trouble for stealing the ring but he didn't touch the necklace because the _ was worth a lot.",
 'option1': 'ring',
 'option2': 'necklace',
 'answer': '1'}

In [3]:
!pip install -U pytorch-pretrained-bert;

     -------------------------------------- 123.8/123.8 kB 1.8 MB/s eta 0:00:00


In [4]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [5]:
class BlankBERT():
    def __init__(self, bert_model):
        self.cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.cuda else "cpu")
        self.bert_model = bert_model

        # get the bert_tokenizer
        self.bert_tokenizer = BertTokenizer.from_pretrained(self.bert_model)

        # build the model
        self.model = BertForMaskedLM.from_pretrained(self.bert_model).to(self.device)
        
        # evaluate the model
        self.model.eval()
        
    def score(self,QuestionTensors, SegmentTensors, MaskedIndex, Candidate):
        # Tokenize the answer
        CandidateTokens = self.bert_tokenizer.tokenize(Candidate)

        # convert token to ids
        CandidateIds = self.bert_tokenizer.convert_tokens_to_ids(CandidateTokens)
        prediction = self.model(QuestionTensors, SegmentTensors)
        PredictionsCandidates = prediction[0,MaskedIndex, CandidateIds].mean()
        return PredictionsCandidates.item()
    
    def predict(self,row):
        # Tokenizing questions
        QuestionTokens = self.bert_tokenizer.tokenize(row['sentence'])
        MaskedIndex = QuestionTokens.index('_')
        # Assign [MASK] in place of  blank 
        QuestionTokens[MaskedIndex] = '[MASK]'
        SegmentIds = [0] * len(QuestionTokens)
        SegmentTensors = torch.tensor([SegmentIds]).to(self.device)
        QuestionIds = self.bert_tokenizer.convert_tokens_to_ids(QuestionTokens)
        QuestionTensors = torch.tensor([QuestionIds]).to(self.device)
        candidates = [row['option1'], row['option2']]
        # get the probabilities of answer options
        PredictTensor = torch.tensor([self.score(QuestionTensors, SegmentTensors,
                                                MaskedIndex, candidate) for candidate in candidates])
        # get the max probability option
        PredictIdx = torch.argmax(PredictTensor).item()
        return candidates[PredictIdx]

In [6]:
Bert_model  = 'bert-large-uncased'
model = BlankBERT(Bert_model)

  5%|███▍                                                            | 66575360/1248501532 [00:34<08:14, 2390451.65B/s]

KeyboardInterrupt: 

In [ ]:
count = 0
from tqdm import tqdm
for question in tqdm(dev):
    anwser_predict = model.predict(question)

    key = 'option'+question['answer']
    if anwser_predict == question[key]:
        count+=1

num_questions = len(dev)
print(f'The model predict {round(count/num_questions,2) * 100} % of total {len(dev)} questions')

In [ ]:
result_submit = []
from tqdm import tqdm
for question in tqdm(test):
    anwser_predict = model.predict(question)

    if question['option1'] == anwser_predict:
      result_submit.append('1')
    else:
      result_submit.append('2')

In [ ]:
fp = open('your_id_pred.txt','w')
for i in result_submit:
  fp.write(i)
  fp.write('\n')
fp.close()